# Data Clean Up

Libraries

In [1]:
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf
import datetime
import numpy as np
import xlwings as xw
from pathlib import Path
import fred_functions as ff

### ETF Daily Returns

Import ETF file

In [2]:
df_returns = pd.read_excel('../df_returns.xlsx',index_col=0,sheet_name='Sheet1')

Select "SPY" and "VBMFX". SPY inception was in 1993 while VBMFX was started in 1986. The data set will cover dates from February 1, 1993 to September 30, 2025

In [3]:
df = df_returns[['SPY','VBMFX']] # select tickets
# df = df.loc['1993-02-01':'2025-09-30'] # filter for dates
df = df.resample("ME").last().dropna(how='all') # get the month end data, dropnas
df_simple_returns = df.pct_change().dropna(how='all') # find the percent change from the prev month, dropnas
df_log_returns = np.log(df/df.shift(1)).dropna(how="all") # calc log returns, just in case we need it

## Macro Indicators (FRED)

import macro File

In [4]:
df_macro = pd.read_excel('../df_macro.xlsx',index_col=0,sheet_name='Sheet1')
# df_macro = df_macro.loc['1990-01-01':'2025-09-30'] # filter for dates

### GDP Data Preprocessing

To prepare quarterly GDP data for our model, we perform a two-step process to handle its frequency and reporting lag:

1.  **Resample to Monthly Frequency:** Since GDP is released **quarterly**, we first resample it to a monthly series. This is done using a **forward-fill** (`ffill`) method, where the GDP value for a given quarter is applied to each month within that quarter. For example, the Q1 GDP value is used for January, February, and March.

2.  **Simulate Reporting Lag:** In practice, GDP figures are not known in real-time (e.g., January's final GDP might not be available until April). To simulate this delay and prevent lookahead bias, we **shift** the resampled monthly data forward. We will create four lagged features: a 1-month lag ,2-month lag, 3-month lag, and a 4-month lag.

In [5]:
df_gdp = ff.forward_fill(df_macro[['RealGDP']].copy())
df_gdp = ff.resample_me(df_gdp)
df_gdp = ff.yoy_change(df_gdp)
df_gdp_lag1 = ff.shift_lag(df_gdp,lag_months=1).rename(columns={"RealGDP":"RealGDP_1m"})
df_gdp_lag2 = ff.shift_lag(df_gdp,lag_months=2).rename(columns={"RealGDP":"RealGDP_2m"})
df_gdp_lag3 = ff.shift_lag(df_gdp,lag_months=3).rename(columns={"RealGDP":"RealGDP_3m"})
df_gdp_lag4 = ff.shift_lag(df_gdp,lag_months=4).rename(columns={"RealGDP":"RealGDP_4m"})
df_gdp_clean = df_gdp.join([df_gdp_lag1, df_gdp_lag2,df_gdp_lag3,df_gdp_lag4], how='inner')

### Permits, CPI, CoreCPI, IndustrialProd, RealRetail


In [6]:
indicator_index_cols = ['Permits','CPI','CoreCPI','IndustrialProd','RealRetail'] # select columns to update
df_indicator_index = df_macro[indicator_index_cols].copy() # make a copy
df_indicator_index = ff.resample_me(df_indicator_index) # get month-end indices
df_lag1_indicator_index = ff.shift_lag(df_indicator_index,lag_months=1) #shift by 1 to account for information lag
# dictionary to rename original numbers to "_lag1"
col_rename_indicator_index = {
    'Permits':'Permits_1m',
    'CPI':'CPI_1m',
    'CoreCPI':'CoreCPI_1m',
    'IndustrialProd':'IndustrialProd_1m',
    'RealRetail':'RealRetail_1m'
}
col_rename_yoy = {
    'Permits_1m':'Permits_1m_yoy',
    'CPI_1m':'CPI_1m_yoy',
    'CoreCPI_1m':'CoreCPI_1m_yoy',
    'IndustrialProd_1m':'IndustrialProd_1m_yoy',
    'RealRetail_1m':'RealRetail_1m_yoy'
}
ff.rename_columns(df_lag1_indicator_index,col_rename_indicator_index) #ff to rename columns using the dict above
df_indicator_yoy = ff.yoy_change(df_lag1_indicator_index) # ff for year over year change
ff.rename_columns(df_indicator_yoy,col_rename_yoy) # rename yoy columns
df_indicator_mom = ff.mom_change(df_lag1_indicator_index) ## ff for month-over-month change
df_indicator_mom = ff.rolling_3m(df_indicator_mom,df_indicator_mom.columns) # ff to calculate 3month rolling avg, and also annualized 3month rolling avg
df_indicator_clean = df_indicator_index.join([df_indicator_yoy,df_indicator_mom], how='inner')

### Unemployment

In [7]:
df_unemployment = df_macro[["UnemploymentRate"]].copy()
df_unemployment = ff.forward_fill(df_unemployment)
df_unemployment = ff.resample_me(df_unemployment)
df_unemployment_1m = ff.shift_lag(df_unemployment,lag_months=1)
df_unemployment_1m = ff.mom_change(df_unemployment_1m)
df_unemployment_clean = ff.rolling_3m(df_unemployment_1m,df_unemployment_1m.columns)

### 

### Rates - Mortgage 30 Year, US 10 YR Treasury, US 2 Year Treasury, US 3 Month Treasury, TIPS10 YR, CorpBAA, CorpAAA, FedFunsRate


In [10]:
ff.credit_spread(df_macro,lag_months=0,delta_calc=True).head(20)

,Mort30Y,UST10Y,UST2Y,UST3M,TIPS10Y,CorpBAA,CorpAAA,FedFundsRate,Slope_10Y_2Y,Slope_10Y_3M,...,TIPS10Y_Delta3m,CorpBAA_Delta3m,CorpAAA_Delta3m,FedFundsRate_Delta3m,Slope_10Y_2Y_Delta3m,Slope_10Y_3M_Delta3m,Slope_10Y_FF_Delta3m,BAA_minus_10Y_Delta3m,AAA_minus_10Y_Delta3m,BAA_minus_AAA_Delta3m
DATE,,,,,,,,,,,,,,,,,,,,,
1985-01-31,12.96,11.17,9.90,8.33,NaN,NaN,11.82,8.73,1.27,2.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985-02-28,12.94,11.91,10.66,8.81,NaN,NaN,12.56,8.74,1.25,3.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985-03-31,13.29,11.65,10.43,8.47,NaN,NaN,12.42,8.58,1.22,3.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985-04-30,13.12,11.41,9.91,8.12,NaN,NaN,12.18,8.58,1.50,3.29,...,NaN,NaN,0.36,-0.15,0.23,0.45,0.39,NaN,0.12,NaN
1985-05-31,12.71,10.28,8.92,7.38,NaN,NaN,11.17,7.64,1.36,2.90,...,NaN,NaN,-1.39,-1.10,0.11,-0.20,-0.53,NaN,0.24,NaN
1985-06-30,12.15,10.25,8.67,7.05,NaN,NaN,10.99,7.95,1.58,3.20,...,NaN,NaN,-1.43,-0.63,0.36,0.02,-0.77,NaN,-0.03,NaN
1985-07-31,12.03,10.57,9.01,7.52,NaN,NaN,11.23,8.61,1.56,3.05,...,NaN,NaN,-0.95,0.03,0.06,-0.24,-0.87,NaN,-0.11,NaN
1985-08-31,12.11,10.28,8.96,7.38,NaN,NaN,10.91,7.80,1.32,2.90,...,NaN,NaN,-0.26,0.16,-0.04,0.00,-0.16,NaN,-0.26,NaN
1985-09-30,12.17,10.31,8.88,7.27,NaN,NaN,11.07,8.84,1.43,3.04,...,NaN,NaN,0.08,0.89,-0.15,-0.16,-0.83,NaN,0.02,NaN
